# Amazon EKS (Elastic Kubernetes Service)

<img src="../_assets/aws_service_icons/eks.svg" width="80" alt="Amazon EKS">

## Goals
- Understand what **Amazon EKS** is
- Know what it’s practically used for
- See a simple **SDK pseudo-code** example (not executed)


## What is EKS?

**Amazon EKS** is AWS’s managed service for running **Kubernetes** clusters.

- AWS manages the **Kubernetes control plane** (API server, etcd, HA, patches).
- You run your workloads on the **data plane**: EC2 worker nodes (managed node groups / self-managed) or **AWS Fargate**.
- It integrates deeply with AWS networking (**VPC**), identity (**IAM**), and load balancing.

Conceptually: EKS gives you a Kubernetes API endpoint that behaves like upstream Kubernetes, while AWS takes responsibility for operating the control plane.


## What is EKS used for (practically)?

EKS is used when you want to run **containerized workloads** with Kubernetes’ scheduling, scaling, and deployment patterns—but prefer a **managed** control plane.

Common use cases:
- **Microservices** and internal platforms (multi-team clusters, namespaces, RBAC)
- **Web APIs** and backends (rolling deploys, autoscaling, service discovery)
- **Batch jobs** and workers (Kubernetes Jobs/CronJobs)
- **ML/DS workflows**: pipeline orchestration, distributed training, and model serving (often alongside tools like Argo, Kubeflow, Ray, MLflow)
- **Hybrid patterns**: consistent deployment model across environments (dev/stage/prod)

Why teams choose it:
- Kubernetes portability + ecosystem
- AWS-managed control plane operations and high availability
- Tight integration with AWS primitives (IAM, VPC, ALB/NLB, CloudWatch)


## Core concepts (minimum you should know)

- **Cluster**: the Kubernetes API + control plane + your worker capacity.
- **Control plane vs data plane**
  - Control plane: managed by AWS.
  - Data plane: your compute (EC2 nodes or Fargate).
- **Node groups**
  - **Managed node group**: AWS manages the lifecycle of worker nodes.
  - Self-managed: you manage nodes and autoscaling.
- **Networking**: cluster lives in a **VPC**; you choose subnets and security groups.
- **IAM & auth**: AWS IAM is used to authenticate to the cluster; Kubernetes RBAC authorizes actions.
- **Add-ons**: critical components like VPC CNI, CoreDNS, kube-proxy (often managed as EKS add-ons).


## Using EKS with an SDK (pseudo-code)

This is **illustrative pseudo-code** showing a common flow using an AWS SDK (e.g., Python `boto3`).

**Do not run as-is**: you must provide a real VPC/subnet setup, IAM roles, region, permissions, and production-grade error handling.

Notes:
- Creating clusters/node groups can incur cost; plan cleanup.
- For real systems, prefer IaC (Terraform/CloudFormation/eksctl) for cluster lifecycle.

```python
# PSEUDO-CODE (do not run)

import boto3
import time

region = "us-east-1"
cluster_name = "demo-eks"
cluster_version = "1.29"

# Pre-created IAM roles (typical requirements)
cluster_role_arn = "arn:aws:iam::<account-id>:role/EKSClusterRole"
node_role_arn = "arn:aws:iam::<account-id>:role/EKSNodeRole"

# Subnets in your VPC where EKS will place control plane ENIs + worker nodes
subnet_ids = ["subnet-aaa", "subnet-bbb"]
security_group_ids = ["sg-123"]

eks = boto3.client("eks", region_name=region)

# 1) Create the EKS cluster (control plane)
eks.create_cluster(
    name=cluster_name,
    version=cluster_version,
    roleArn=cluster_role_arn,
    resourcesVpcConfig={
        "subnetIds": subnet_ids,
        "securityGroupIds": security_group_ids,
        "endpointPublicAccess": True,
        # "endpointPrivateAccess": True,
    },
)

# 2) Wait until the cluster is ACTIVE
while True:
    status = eks.describe_cluster(name=cluster_name)["cluster"]["status"]
    if status == "ACTIVE":
        break
    time.sleep(30)

# 3) Add worker capacity via a managed node group
eks.create_nodegroup(
    clusterName=cluster_name,
    nodegroupName="ng-1",
    nodeRole=node_role_arn,
    subnets=subnet_ids,
    instanceTypes=["m5.large"],
    scalingConfig={"minSize": 1, "desiredSize": 2, "maxSize": 4},
)

# 4) Get connection info for kubectl / Kubernetes client
cluster = eks.describe_cluster(name=cluster_name)["cluster"]
endpoint = cluster["endpoint"]
ca_data = cluster["certificateAuthority"]["data"]

# 5) Deploy workloads to Kubernetes
# Typically done with kubectl, GitOps, or a Kubernetes client library.
kubeconfig = build_kubeconfig(
    cluster_name=cluster_name,
    endpoint=endpoint,
    certificate_authority_data=ca_data,
    auth_via_iam=True,
)

k8s = KubernetesClient(kubeconfig)
k8s.apply_yaml("k8s/deployment.yaml")
```


## Pitfalls & quick tips

- Plan **VPC/subnets** up front (private vs public, NAT, routing).
- Be explicit about **IAM roles** and how you map IAM identities to Kubernetes RBAC.
- Keep an eye on **Kubernetes version upgrades** and add-on compatibility.
- For production, use GitOps/CI workflows to deploy manifests instead of “manual kubectl”.

## References
- AWS Docs: Amazon EKS
- AWS Docs: EKS API (CreateCluster, CreateNodegroup, DescribeCluster)
